# Titanic prediction

In [1]:
# Titanic prediction

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
path = "/content/drive/MyDrive/Colab Notebooks/ML CW/titanic.csv"
data = pd.read_csv(path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/ML CW/titanic.csv'

## Understand data

In [ ]:
data.shape
data = data.drop(["PassengerId"], axis="columns")

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
corr = data.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.show()

In [ ]:
# compare survival rate across Age, SibSp, Parch, and Fare
pd.pivot_table(data, index = 'Survival', values = ['Age','SibSp','Parch','Fare'])

In [ ]:
import pandas as pd

print(pd.pivot_table(data, index='Survival', columns='Pclass', values='Ticket', aggfunc='count'))
print()
print(pd.pivot_table(data, index='Survival', columns='Sex', values='Ticket', aggfunc='count'))
print()
print(pd.pivot_table(data, index='Survival', columns='Embarked', values='Ticket', aggfunc='count'))


In [ ]:
data.hist(bins=50, figsize=(20, 10))

# normalize distribution

In [ ]:
# preprocess
# fare
# sibsp
# parch
import numpy as np
data['Parch'] = np.log(data['Parch'] + 1)
data['SibSp'] = np.log(data['SibSp'] + 1)
data['Fare'] = np.log(data['Fare'] + 1)
data.hist(bins=50, figsize=(20, 10), column=['Parch', 'SibSp', 'Fare'])


## feature engineering

Tickets - Do different ticket types impact survival rates?

Does a person's title relate to survival rates

In [ ]:
# #feature engineering on person's title
# data.Name.head(50)
# data['name_title'] = data.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
# #mr., ms., master. etc
# data['name_title'].value_counts()
print(data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip()).value_counts())


In [ ]:
# #understand ticket values better
# #numeric vs non numeric
# data['numeric_ticket'] = data.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
# data['ticket_letters'] = data.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)

# data['numeric_ticket'].value_counts()

# data cleaning

## impute values as we would lose a lot of data

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class AgeImputer(BaseEstimator, TransformerMixin):
  def fit(self, x, y=None):
    return self

  def transform(self, X):
    imputer = SimpleImputer(strategy="mean")
    X["Age"] = imputer.fit_transform(X[['Age']])
    # X["Pclass"] = imputer.fit_transform(X[['Pclass']])
    # X["Embarked"] = imputer.fit_transform(X[['Embarked']])
    return X

## categorical data

In [ ]:
data["Sex"].value_counts()

In [ ]:
data["Embarked"].value_counts()

In [ ]:
from sklearn.preprocessing import OneHotEncoder


class FeatureEncoder(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, X):
      encoder = OneHotEncoder()
      matrix = encoder.fit_transform(X[['Embarked']]).toarray()
      column_names = ["C", "S", "Q", "N"]

      for i in range(len(matrix.T)):
          X[column_names[i]] = matrix.T[i]

      matrix = encoder.fit_transform(X[['Sex']]).toarray()
      column_names = ["female", "male"]

      for i in range(len(matrix.T)):
          X[column_names[i]] = matrix.T[i]

      return X


In [ ]:
class FeatureDropper(BaseEstimator, TransformerMixin):
  def fit(self, x, y=None):
    return self

  def transform(self, x):
    return x.drop(["Embarked", "Sex", "Name", "Ticket", "N"], axis=1, errors="ignore")


In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

# Initialize the imputer to replace missing values with the median
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

# Fit the imputer to the 'Pclass' column
data['Pclass'] = imputer.fit_transform(data[['Pclass']])


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([("ageimputer", AgeImputer()),
                     ("featureencoder", FeatureEncoder()),
                     ("featuredropper", FeatureDropper())])

In [ ]:
data = pipeline.fit_transform(data)
# data.dropna(inplace=True)


In [ ]:
data

In [ ]:
data.info()

In [ ]:
plt.figure(figsize=(15, 8))
sns.heatmap(data.corr(), annot=True, cmap="YlGnBu")

# Model seletion

## scale data

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assuming your pipeline has already been fitted and transformed the data
# and the 'data' variable is the DataFrame ready to be split and scaled

# Separate features and target variable if they are in the same DataFrame
# This step is dependent on your dataset.
# Let's say the target column is named 'Target'.
x = data.drop('Survival', axis=1)  # features
y = data['Survival']  # target variable

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both the training and testing data
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Now X_train_scaled and X_test_scaled contain the scaled features
# y_train and y_test contain the corresponding target variable data


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
lr = LogisticRegression(max_iter = 2000)
cv = cross_val_score(lr,x_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
rf = RandomForestClassifier(random_state = 1)
cv = cross_val_score(rf,x_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
svc = SVC(probability = True)
cv = cross_val_score(svc,x_train,y_train,cv=5)
print(cv)
print(cv.mean())

# random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier()

param_grid = [
    {"n_estimators": [10, 100, 200, 500], "max_depth": [None, 5, 10], "min_samples_split": [2,3,4]}
]

grid_search = GridSearchCV(clf, param_grid, cv=3, scoring='accuracy', return_train_score=True)
grid_search.fit(x_train, y_train)


In [ ]:
final_clf = grid_search.best_estimator_

In [ ]:
final_clf

In [ ]:
# Evaluate the best estimator on the test set
test_score = final_clf.score(x_test, y_test)

# Print the score of the best estimator
print("Score of the best estimator on the test set:", test_score)


# Model tune performance

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Updated performance reporting function for classification models
def clf_performance(classifier, model_name, x_test, y_test):
    y_pred = classifier.predict(x_test)

    print(model_name)
    print('Best Score (mean cross-validated score of the best_estimator): ' + str(classifier.best_score_))
    print('Best Parameters: ' + str(classifier.best_params_))

    # Classification Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='binary')
    recall = recall_score(y_test, y_pred, average='binary')
    f1 = f1_score(y_test, y_pred, average='binary')
    cm = confusion_matrix(y_test, y_pred)

    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')
    print('Confusion Matrix:\n', cm)

# Example usage:
# Assume you've already fit your classifier with best_clf_lr = clf_lr.fit(x_train, y_train)
# clf_performance(best_clf_lr, 'Logistic Regression', x_test, y_test)

In [ ]:
lr = LogisticRegression()
param_grid = {'max_iter' : [2000],
              'penalty' : ['l1', 'l2'],
              'C' : np.logspace(-4, 4, 20),
              'solver' : ['liblinear']}

clf_lr = GridSearchCV(lr, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_lr = clf_lr.fit(x_train,y_train)
clf_performance(best_clf_lr,'Logistic Regression',  x_test, y_test)

In [ ]:
rf = RandomForestClassifier(random_state = 1)
param_grid =  {'n_estimators': [400,450,500,550],
               'criterion':['gini','entropy'],
                                  'bootstrap': [True],
                                  'max_depth': [15, 20, 25],
                                  'max_features': ['auto','sqrt', 10],
                                  'min_samples_leaf': [2,3],
                                  'min_samples_split': [2,3]}

clf_rf = GridSearchCV(rf, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_rf = clf_rf.fit(x_train,y_train)
clf_performance(best_clf_rf,'Random Forest',  x_test, y_test)

In [ ]:
svc = SVC(probability = True)
param_grid = tuned_parameters = [{'kernel': ['rbf'], 'gamma': [.1,.5,1,2,5,10],
                                  'C': [.1, 1, 10, 100, 1000]},
                                 {'kernel': ['linear'], 'C': [.1, 1, 10, 100, 1000]},
                                 {'kernel': ['poly'], 'degree' : [2,3,4,5], 'C': [.1, 1, 10, 100, 1000]}]
clf_svc = GridSearchCV(svc, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_svc = clf_svc.fit(x_train,y_train)
clf_performance(best_clf_svc,'SVC',  x_test, y_test)

In [ ]:
#Because the total feature space is so large, I used a randomized search to narrow down the paramters for the model. I took the best model from this and did a more granular search
"""
rf = RandomForestClassifier(random_state = 1)
param_grid =  {'n_estimators': [100,500,1000],
                                  'bootstrap': [True,False],
                                  'max_depth': [3,5,10,20,50,75,100,None],
                                  'max_features': ['auto','sqrt'],
                                  'min_samples_leaf': [1,2,4,10],
                                  'min_samples_split': [2,5,10]}

clf_rf_rnd = RandomizedSearchCV(rf, param_distributions = param_grid, n_iter = 100, cv = 5, verbose = True, n_jobs = -1)
best_clf_rf_rnd = clf_rf_rnd.fit(X_train_scaled,y_train)
clf_performance(best_clf_rf_rnd,'Random Forest')"""

In [ ]:
best_rf = best_clf_rf.best_estimator_.fit(x_train,y_train)
feat_importances = pd.Series(best_rf.feature_importances_, index=x_train.columns)
feat_importances.nlargest(20).plot(kind='barh')